In [1]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import time

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)

start_time = time.time()
data=pd.read_csv('covtype.csv')
print("Read time = {} seconds".format(time.time() - start_time))

Read time = 2.0254478454589844 seconds


In [3]:
print('Data Dimension:')
print('Number of Records:', data.shape[0])
print('Number of Features:', data.shape[1])

Data Dimension:
Number of Records: 581012
Number of Features: 55


In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [5]:
data.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,0.448865,0.051434,0.436074,0.063627,0.005217,0.012952,0.008301,0.021335,0.002749,0.011316,0.000181,0.000308,0.001974,0.056168,0.021359,0.051584,0.030001,0.001031,0.000005,0.004897,0.005890,0.003268,0.006921,0.015936,0.001442,0.057439,0.099399,0.036622,0.000816,0.004456,0.001869,0.001628,0.198356,0.051927,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,0.497379,0.220882,0.495897,0.244087,0.072039,0.113066,0.090731,0.144499,0.052356,0.105775,0.013442,0.017550,0.044387,0.230245,0.144579,0.221186,0.170590,0.032092,0.002272,0.069804,0.076518,0.057077,0.082902,0.125228,0.037950,0.232681,0.299197,0.187833,0.028551,0.066605,0.043193,0.040318,0.398762,0.221879,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [6]:
from sklearn.model_selection import train_test_split

target = data.Cover_Type
train = data.drop(['Cover_Type'], axis = 1)
# models_results = dict()

start_time = time.time()
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.25, random_state=16)
print("Train test split time = {} seconds".format(time.time() - start_time))

Train test split time = 0.5486798286437988 seconds


In [7]:
from sklearn.ensemble import ExtraTreesClassifier

trees = ExtraTreesClassifier(n_estimators = 100, max_features=10)
start_time = time.time()
trees.fit(x_train, y_train)
print("ExtraTreesClassifier train time = {} seconds".format(time.time() - start_time))

ExtraTreesClassifier train time = 86.6052839756012 seconds


In [8]:
start_time = time.time()
trees_result = trees.predict(x_test)
print("ExtraTreesClassifier predict time = {} seconds".format(time.time() - start_time))

ExtraTreesClassifier predict time = 8.230011940002441 seconds


In [9]:
start_time = time.time()
extraTreesClassifier_accuracy = accuracy_score(y_test, trees_result)
print("ExtraTreesClassifier accuracy time = {} seconds".format(time.time() - start_time))
print("ExtraTreesClassifier accuracy: ", extraTreesClassifier_accuracy)

ExtraTreesClassifier accuracy time = 0.008679866790771484 seconds
ExtraTreesClassifier accuracy:  0.9578046580793512


In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn_neighbors = []
knn_results = []
for neighbors in range(5, 6):
    knn_neighbors.append(neighbors)
    knn = KNeighborsClassifier(n_neighbors=neighbors)
#     start_time = time.time()
    knn.fit(x_train, y_train)
#     print("KNeighborsClassifier{} train time = {} seconds".format(neighbors, time.time() - start_time))

#     start_time = time.time()
    knn_predict_result = knn.predict(x_test)
#     print("KNeighborsClassifier{} predict time = {} seconds".format(neighbors, time.time() - start_time))
    
#     start_time = time.time()
    accuracy = accuracy_score(y_test, knn_predict_result)
#     print("KNeighborsClassifier{} accuracy time = {} seconds".format(neighbors, time.time() - start_time))
#     print("KNeighborsClassifier{} accuracy".format(neighbors), ": ", accuracy)

    
    knn_results.append(accuracy)

In [11]:
for index in range(len(knn_neighbors)):
    print("Neighbors={}, accuracy={}".format(knn_neighbors[index], knn_results[index]))

Neighbors=5, accuracy=0.9676151267099474


In [12]:
import xgboost

xgboost_model = xgboost.XGBClassifier()
start_time = time.time()
xgboost_model.fit(x_train, y_train)
print("XGBClassifier train time = {} seconds".format(time.time() - start_time))

XGBClassifier train time = 477.46629881858826 seconds


In [13]:
start_time = time.time()
xgboost_result = xgboost_model.predict(x_test)
print("XGBoost predict time = {} seconds".format(time.time() - start_time))

start_time = time.time()
xgboost_model_accuracy = accuracy_score(y_test, xgboost_result)
print("XGBoost accuracy time = {} seconds".format(time.time() - start_time))
print("XGBoost accuracy: ", xgboost_model_accuracy)

XGBoost predict time = 2.3058087825775146 seconds
XGBoost accuracy time = 0.00832819938659668 seconds
XGBoost accuracy:  0.8691868670526598


In [14]:
from sklearn.ensemble import RandomForestClassifier

randomForestClassifier = RandomForestClassifier(n_estimators = 100, max_features=10)
start_time = time.time()
randomForestClassifier.fit(x_train, y_train)
print("RandomForestClassifier train time = {} seconds".format(time.time() - start_time))

RandomForestClassifier train time = 150.61444902420044 seconds


In [15]:
start_time = time.time()
randomForest_result = randomForestClassifier.predict(x_test)
print("RandomForestClassifier predict time = {} seconds".format(time.time() - start_time))

start_time = time.time()
randomForest_accuracy = accuracy_score(y_test, randomForest_result)
print("RandomForestClassifier accuracy time = {} seconds".format(time.time() - start_time))
print("RandomForestClassifier accuracy: ", randomForest_accuracy)

RandomForestClassifier predict time = 5.32769513130188 seconds
RandomForestClassifier accuracy time = 0.0064198970794677734 seconds
RandomForestClassifier accuracy:  0.959477601151095


In [16]:
# from sklearn.svm import SVC

# svclassifier = SVC(kernel='linear')
# start_time = time.time()
# svclassifier.fit(x_train, y_train)
# print("SVC linear train time = {} seconds".format(time.time() - start_time))

In [17]:
# start_time = time.time()
# print("SVC accuracy: ", accuracy_score(y_test, [round(item) for item in svclassifier.predict(x_test)]))
# print("SVC accuracy time = {} seconds".format(time.time() - start_time))

In [18]:
# start_time = time.time()
# print("SVC score: ", svclassifier.score(x_test, y_test))
# print("SVC score time = {} seconds".format(time.time() - start_time))